In [75]:
import warnings

# 오류 경고 무시하기
warnings.filterwarnings(action='ignore')

# 1. 데이터 불러오기

In [76]:
import pandas as pd
import os

pd.options.display.float_format = '{:.3f}'.format
pd.set_option('display.max_columns', None)

PATH = '../data/'

file_list= os.listdir(PATH)
csv_list = list()

for file in file_list:
    if file.split(".")[-1] == 'csv':
        csv_list.append(file)
data = ['living_popul', 'resident_popul', 'area', 'store', 'working_popul', 'facilitie', 'estimated_sales']

for name, file in zip(data, csv_list):
    globals()[name] = pd.read_csv(PATH + file, encoding = "ansi")

store = store.drop("Unnamed: 0", axis = 1)
estimated_sales = estimated_sales.drop("Unnamed: 0", axis = 1)

# 2. 데이터 전처리

## 2-0. 시점 통일 / 관광특구 제거

In [77]:
# 시점 통일
years = [20223, 20224, 20231, 20232]

living_popul = living_popul[living_popul['기준_년분기_코드'].isin(years)]
resident_popul = resident_popul[resident_popul['기준_년분기_코드'].isin(years)]
store = store[store['기준_년분기_코드'].isin(years)]
working_popul = working_popul[working_popul['기준_년분기_코드'].isin(years)]
facilitie = facilitie[facilitie['기준_년분기_코드'].isin(years)]
estimated_sales = estimated_sales[estimated_sales['기준_년분기_코드'].isin(years)]

# 관광특구 제거
living_popul = living_popul[living_popul['상권_구분_코드_명']!='관광특구']
resident_popul = resident_popul[resident_popul['상권_구분_코드_명']!='관광특구']
store = store[store['상권_구분_코드_명']!='관광특구']
working_popul = working_popul[working_popul['상권_구분_코드_명']!='관광특구']
facilitie = facilitie[facilitie['상권_구분_코드_명']!='관광특구']
estimated_sales = estimated_sales[estimated_sales['상권_구분_코드_명']!='관광특구']

## 2-1. 길단위인구 전처리

### 2-1-1 변수 처리

In [78]:
# 파생변수 생성
living_popul['시간대_21_06_유동인구_수'] = living_popul['시간대_00_06_유동인구_수'] + living_popul['시간대_21_24_유동인구_수']

living_popul['주중_유동인구_수'] = living_popul['월요일_유동인구_수'] + living_popul['화요일_유동인구_수'] + living_popul['수요일_유동인구_수'] + living_popul['목요일_유동인구_수'] + living_popul['금요일_유동인구_수']
living_popul['주말_유동인구_수'] = living_popul['토요일_유동인구_수'] + living_popul['일요일_유동인구_수']

living_popul['연령대_30_40_유동인구_수'] = living_popul['연령대_30_유동인구_수'] + living_popul['연령대_40_유동인구_수']
living_popul['연령대_50_60_유동인구_수'] = living_popul['연령대_50_유동인구_수'] + living_popul['연령대_60_이상_유동인구_수']

# 불필요한 변수 제거
living_popul = living_popul.drop(['시간대_00_06_유동인구_수', '시간대_21_24_유동인구_수',
                                  '월요일_유동인구_수', '화요일_유동인구_수', '수요일_유동인구_수', '목요일_유동인구_수', '금요일_유동인구_수', '토요일_유동인구_수', '일요일_유동인구_수', 
                                  '연령대_30_유동인구_수','연령대_40_유동인구_수','연령대_50_유동인구_수','연령대_60_이상_유동인구_수'], axis = 1)

### 2-1-2. 결측치/이상치/중복

In [79]:
# 결측값 확인
living_popul.isnull().sum()

기준_년분기_코드           0
상권_구분_코드_명          0
상권_코드_명             0
총_유동인구_수            0
남성_유동인구_수           0
여성_유동인구_수           0
연령대_10_유동인구_수       0
연령대_20_유동인구_수       0
시간대_06_11_유동인구_수    0
시간대_11_14_유동인구_수    0
시간대_14_17_유동인구_수    0
시간대_17_21_유동인구_수    0
시간대_21_06_유동인구_수    0
주중_유동인구_수           0
주말_유동인구_수           0
연령대_30_40_유동인구_수    0
연령대_50_60_유동인구_수    0
dtype: int64

In [80]:
# 중복 확인
living_popul[living_popul.duplicated()]

,기준_년분기_코드,상권_구분_코드_명,상권_코드_명,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,시간대_06_11_유동인구_수,시간대_11_14_유동인구_수,시간대_14_17_유동인구_수,시간대_17_21_유동인구_수,시간대_21_06_유동인구_수,주중_유동인구_수,주말_유동인구_수,연령대_30_40_유동인구_수,연령대_50_60_유동인구_수


In [81]:
# 통계량 확인
living_popul.describe()

,기준_년분기_코드,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,시간대_06_11_유동인구_수,시간대_11_14_유동인구_수,시간대_14_17_유동인구_수,시간대_17_21_유동인구_수,시간대_21_06_유동인구_수,주중_유동인구_수,주말_유동인구_수,연령대_30_40_유동인구_수,연령대_50_60_유동인구_수
count,6573.000,6573.000,6573.000,6573.000,6573.000,6573.000,6573.000,6573.000,6573.000,6573.000,6573.000,6573.000,6573.000,6573.000,6573.000
mean,20227.499,819172.761,390059.275,429113.498,105750.179,146463.772,166764.988,104697.384,105734.445,141765.368,300210.649,590223.923,228948.919,276078.394,290880.488
std,4.032,860738.310,415405.204,448793.459,112986.973,211285.817,172023.931,119372.114,122503.528,158734.038,318552.356,625430.046,240185.552,311897.390,293363.299
min,20223.000,12.000,12.000,0.000,0.000,0.000,4.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,20223.000,221071.000,106737.000,113709.000,25585.000,32148.000,45786.000,28449.000,28568.000,38153.000,75518.000,158928.000,60054.000,74201.000,79250.000
50%,20224.000,566885.000,269133.000,296501.000,71313.000,82738.000,116615.000,71878.000,71921.000,95899.000,205337.000,405241.000,158567.000,184591.000,205254.000
75%,20231.000,1130788.000,534060.000,588669.000,146355.000,182991.000,233054.000,139191.000,139491.000,193507.000,414441.000,822348.000,311102.000,369148.000,409490.000
max,20232.000,7942915.000,4000287.000,4032096.000,829062.000,3487417.000,1529921.000,1429461.000,1491003.000,1793125.000,2942275.000,6236830.000,2124591.000,3433420.000,2463853.000


### 2-1-3. 분기별 데이터 연도 기준 변환

In [82]:
living_popul_year = living_popul.drop('기준_년분기_코드', axis = 1)\
                                .groupby(['상권_구분_코드_명','상권_코드_명'],as_index = False)\
                                .mean()

## 2-2. 상주인구 전처리

### 2-2-1. 변수 처리

In [83]:
# 파생변수 생성
resident_popul['연령대_30_40_상주인구_수'] = resident_popul['연령대_30_상주인구_수']+resident_popul['연령대_40_상주인구_수']
resident_popul['연령대_50_60_상주인구_수'] = resident_popul['연령대_50_상주인구_수']+resident_popul['연령대_60_이상_상주인구_수']

# 불필요한 변수 제거
resident_popul = resident_popul[['기준_년분기_코드', '상권_구분_코드_명', '상권_코드_명',
                                 '총_상주인구_수', '남성_상주인구_수','여성_상주인구_수', 
                                 '연령대_10_상주인구_수', '연령대_20_상주인구_수', '연령대_30_40_상주인구_수', '연령대_50_60_상주인구_수']]

### 2-2-2. 결측치/이상치/중복

In [84]:
# 결측값 확인
resident_popul.isnull().sum()

기준_년분기_코드           0
상권_구분_코드_명          0
상권_코드_명             0
총_상주인구_수            0
남성_상주인구_수           0
여성_상주인구_수           0
연령대_10_상주인구_수       0
연령대_20_상주인구_수       0
연령대_30_40_상주인구_수    0
연령대_50_60_상주인구_수    0
dtype: int64

In [85]:
# 중복 확인
resident_popul[resident_popul.duplicated()]

,기준_년분기_코드,상권_구분_코드_명,상권_코드_명,총_상주인구_수,남성_상주인구_수,여성_상주인구_수,연령대_10_상주인구_수,연령대_20_상주인구_수,연령대_30_40_상주인구_수,연령대_50_60_상주인구_수


In [86]:
# 통계량 확인
resident_popul.describe()

,기준_년분기_코드,총_상주인구_수,남성_상주인구_수,여성_상주인구_수,연령대_10_상주인구_수,연령대_20_상주인구_수,연령대_30_40_상주인구_수,연령대_50_60_상주인구_수
count,6488.000,6488.000,6488.000,6488.000,6488.000,6488.000,6488.000,6488.000
mean,20227.495,2359.471,1165.882,1193.589,230.282,300.194,789.012,1039.983
std,4.032,2348.110,1159.129,1194.338,262.202,339.598,833.507,1031.546
min,20223.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,20223.000,657.000,326.750,330.750,53.000,70.000,207.000,280.000
50%,20224.000,1614.000,809.000,817.000,147.000,195.000,528.000,720.000
75%,20231.000,3305.000,1639.000,1677.000,318.000,413.250,1083.000,1464.250
max,20232.000,21341.000,10459.000,10882.000,3078.000,4457.000,7626.000,8342.000


### 2-2-3. 분기별 데이터 연도 기준 변환

In [87]:
resident_popul_year = resident_popul[resident_popul['기준_년분기_코드'] == 20232].drop(['기준_년분기_코드'],axis=1)

## 2-3. 점포 전처리

### 2-3-1. 변수 처리

In [88]:
# 불필요한 변수 삭제
store.drop(columns = ['상권_구분_코드', '상권_코드'], inplace = True)

### 2-3-1. 결측치/이상치/중복

In [89]:
# 결측값 확인
store.isnull().sum()

기준_년분기_코드      0
상권_구분_코드_명     0
상권_코드_명        0
서비스_업종_코드      0
서비스_업종_코드_명    0
점포_수           0
유사_업종_점포_수     0
개업_율           0
개업_점포_수        0
폐업_률           0
폐업_점포_수        0
프랜차이즈_점포_수     0
dtype: int64

In [90]:
# 중복 확인
store[store.duplicated()]

,기준_년분기_코드,상권_구분_코드_명,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수


In [91]:
# 통계량 확인
store.describe()

,기준_년분기_코드,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수
count,302726.000,302726.000,302726.000,302726.000,302726.000,302726.000,302726.000,302726.000
mean,20227.497,5.692,6.126,2.296,0.134,2.482,0.171,0.434
std,4.031,25.823,26.281,10.924,0.584,11.827,0.784,2.047
min,20223.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,20223.000,1.000,1.000,0.000,0.000,0.000,0.000,0.000
50%,20224.000,2.000,2.000,0.000,0.000,0.000,0.000,0.000
75%,20231.000,4.000,5.000,0.000,0.000,0.000,0.000,0.000
max,20232.000,2190.000,2191.000,200.000,28.000,400.000,71.000,123.000


### 2-3-2. 분기별 데이터 연도 기준 변환

In [92]:
store_year = store[store['기준_년분기_코드'] == 20232].drop(['기준_년분기_코드', '서비스_업종_코드'], axis = 1)\
                                                    .groupby(['상권_구분_코드_명', '상권_코드_명', '서비스_업종_코드_명'], as_index = False)\
                                                    .sum()

## 2-4. 직장인구 전처리

### 2-4-1. 변수 처리

In [93]:
working_popul = working_popul[['기준_년분기_코드', '상권_구분_코드_명', '상권_코드_명', '총_직장_인구_수']]

### 2-4-2. 결측치/이상치/중복

In [94]:
# 결측값 확인
working_popul.isnull().sum()

기준_년분기_코드     0
상권_구분_코드_명    0
상권_코드_명       0
총_직장_인구_수     0
dtype: int64

In [95]:
# 중복 확인
working_popul[working_popul.duplicated()]

,기준_년분기_코드,상권_구분_코드_명,상권_코드_명,총_직장_인구_수


In [96]:
# 통계량 확인
working_popul.describe()

,기준_년분기_코드,총_직장_인구_수
count,6468.000,6468.000
mean,20227.500,1660.112
std,4.031,5611.955
min,20223.000,1.000
25%,20223.750,105.000
50%,20227.500,315.000
75%,20231.250,876.000
max,20232.000,81127.000


### 2-4-3. 분기별 데이터 연도 기준 변환

In [97]:
working_popul_year = working_popul[working_popul['기준_년분기_코드'] == 20232].drop(['기준_년분기_코드'],axis=1)

## 2-5. 집객시설 전처리

### 2-5-1. 변수 처리

In [98]:
# 파생변수 생성
facilitie['초중고_수'] = facilitie['초등학교_수']+facilitie['중학교_수']+facilitie['고등학교_수']

facilitie['교통시설수'] = facilitie['버스_정거장_수'] + facilitie['지하철_역_수']
 
facilitie['집객시설수'] = facilitie['집객시설_수']-facilitie['교통시설수']

# 불필요한 변수 제거
facilitie = facilitie.drop(['집객시설_수', '철도_역_수', '초등학교_수', '중학교_수', '고등학교_수', '버스_정거장_수', '지하철_역_수'], axis=1)
facilitie = facilitie[['기준_년분기_코드', '상권_구분_코드_명', '상권_코드_명', '집객시설수', '교통시설수' ]]

### 2-5-2. 결측치/이상치/중복

In [99]:
# 결측값 확인
facilitie.isnull().sum()

기준_년분기_코드        0
상권_구분_코드_명       0
상권_코드_명          0
집객시설수         5556
교통시설수         5556
dtype: int64

In [100]:
# 결측값 0으로 대체
facilitie = facilitie.fillna(0)

In [101]:
# 중복 확인
facilitie[facilitie.duplicated()]

,기준_년분기_코드,상권_구분_코드_명,상권_코드_명,집객시설수,교통시설수


In [102]:
# 통계량 확인
facilitie.describe()

,기준_년분기_코드,집객시설수,교통시설수
count,6288.000,6288.000,6288.000
mean,20227.500,6.245,1.243
std,4.031,23.680,4.459
min,20223.000,0.000,0.000
25%,20223.750,0.000,0.000
50%,20227.500,0.000,0.000
75%,20231.250,0.000,0.000
max,20232.000,351.000,87.000


### 2-5-3. 분기별 데이터 연도 기준 변환

In [103]:
facilitie_year = facilitie[facilitie['기준_년분기_코드'] == 20232].drop(['기준_년분기_코드'], axis = 1)

## 2-6. 추정매출 전처리

### 2-6-1. 변수 처리

In [104]:
# 파생변수 생성
def sales_tran(df):
    if df['당월_법인_매출_금액'] < 0:
        return df['당월_매출_금액'] - df['당월_법인_매출_금액']
    else :
        return df['당월_매출_금액']

estimated_sales['당월_개인_매출_금액'] = estimated_sales['남성_매출_금액'] + estimated_sales['여성_매출_금액']
estimated_sales['당월_법인_매출_금액'] =  estimated_sales['당월_매출_금액'] - estimated_sales['당월_개인_매출_금액']

estimated_sales['당월_매출_금액'] = estimated_sales.apply(lambda x : sales_tran(x) , axis=1)

estimated_sales.loc[estimated_sales['당월_법인_매출_금액']<0, '당월_법인_매출_금액'] = 0
estimated_sales['시간대_21~06_매출_금액'] = estimated_sales['시간대_00~06_매출_금액'] + estimated_sales['시간대_21~24_매출_금액']
estimated_sales['시간대_21~06_매출_건수'] = estimated_sales['시간대_건수~06_매출_건수'] + estimated_sales['시간대_건수~24_매출_건수']
estimated_sales['연령대_30_40_매출_금액'] = estimated_sales['연령대_30_매출_금액'] + estimated_sales['연령대_40_매출_금액']
estimated_sales['연령대_50_60_매출_금액'] = estimated_sales['연령대_50_매출_금액'] + estimated_sales['연령대_60_이상_매출_금액']
estimated_sales['연령대_30_40_매출_건수'] = estimated_sales['연령대_30_매출_건수'] + estimated_sales['연령대_40_매출_건수']
estimated_sales['연령대_50_60_매출_건수'] = estimated_sales['연령대_50_매출_건수'] + estimated_sales['연령대_60_이상_매출_건수']

# 불필요한 변수 제거
estimated_sales = estimated_sales.drop(columns = ['서비스_업종_코드'],
                                       axis = 1)

estimated_sales = estimated_sales.drop(columns = ['월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액',
                                                  '월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수'],
                                       axis = 1)
estimated_sales = estimated_sales.drop(columns = ['시간대_00~06_매출_금액', '시간대_21~24_매출_금액', '시간대_건수~06_매출_건수', '시간대_건수~24_매출_건수',
                                                 '연령대_30_매출_금액','연령대_40_매출_금액','연령대_50_매출_금액','연령대_60_이상_매출_금액',
                                                 '연령대_30_매출_건수','연령대_40_매출_건수','연령대_50_매출_건수','연령대_60_이상_매출_건수'],
                                       axis = 1)

# 변수 이름 변경
estimated_sales = estimated_sales.rename(columns = {'시간대_건수~11_매출_건수' : '시간대_06~11_매출_건수',
                                                         '시간대_건수~14_매출_건수' : '시간대_11~14_매출_건수',
                                                         '시간대_건수~17_매출_건수' : '시간대_14~17_매출_건수',
                                                         '시간대_건수~21_매출_건수' : '시간대_17~21_매출_건수'})



### 2-6-2. 결측치/이상치/중복

In [105]:
# 결측값 확인
estimated_sales.isnull().sum()

기준_년분기_코드          0
상권_구분_코드           0
상권_구분_코드_명         0
상권_코드              0
상권_코드_명            0
서비스_업종_코드_명        0
당월_매출_금액           0
당월_매출_건수           0
주중_매출_금액           0
주말_매출_금액           0
시간대_06~11_매출_금액    0
시간대_11~14_매출_금액    0
시간대_14~17_매출_금액    0
시간대_17~21_매출_금액    0
남성_매출_금액           0
여성_매출_금액           0
연령대_10_매출_금액       0
연령대_20_매출_금액       0
주중_매출_건수           0
주말_매출_건수           0
시간대_06~11_매출_건수    0
시간대_11~14_매출_건수    0
시간대_14~17_매출_건수    0
시간대_17~21_매출_건수    0
남성_매출_건수           0
여성_매출_건수           0
연령대_10_매출_건수       0
연령대_20_매출_건수       0
당월_개인_매출_금액        0
당월_법인_매출_금액        0
시간대_21~06_매출_금액    0
시간대_21~06_매출_건수    0
연령대_30_40_매출_금액    0
연령대_50_60_매출_금액    0
연령대_30_40_매출_건수    0
연령대_50_60_매출_건수    0
dtype: int64

In [106]:
# 통계량 확인
estimated_sales.describe()

,기준_년분기_코드,상권_코드,당월_매출_금액,당월_매출_건수,주중_매출_금액,주말_매출_금액,시간대_06~11_매출_금액,시간대_11~14_매출_금액,시간대_14~17_매출_금액,시간대_17~21_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,주중_매출_건수,주말_매출_건수,시간대_06~11_매출_건수,시간대_11~14_매출_건수,시간대_14~17_매출_건수,시간대_17~21_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,당월_개인_매출_금액,당월_법인_매출_금액,시간대_21~06_매출_금액,시간대_21~06_매출_건수,연령대_30_40_매출_금액,연령대_50_60_매출_금액,연령대_30_40_매출_건수,연령대_50_60_매출_건수
count,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000,83142.000
mean,20227.506,3116851.531,1057530630.270,37378.921,800998140.292,256532440.674,112812620.583,270639855.442,263642589.533,289522260.989,482553877.888,454435667.863,5388979.464,125426918.046,28218.137,9160.785,5231.569,9703.087,7869.907,9959.027,18572.037,17223.529,559.116,7452.457,936989545.752,120541084.519,120913224.832,4615.330,406737845.752,399433563.022,15695.358,12088.637
std,4.031,7072.715,9247199458.539,158825.344,6375468667.307,3472245561.224,1170991051.609,2448384035.026,3090139741.176,2707954382.716,4423202984.139,3474196467.177,29279037.589,839832564.084,111071.872,54745.766,27854.478,43249.044,41592.007,44177.137,80157.083,76442.682,3521.526,39113.158,7676380836.375,2380499063.105,983837619.257,22925.971,3379770530.233,3754679997.219,67955.953,60479.077
min,20223.000,3110001.000,12.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,20224.000,3110566.000,48000000.000,859.000,35761235.750,7349956.500,135702.250,5399863.500,7746083.500,11697304.250,19337881.500,17810972.500,0.000,1333739.750,646.000,152.000,6.000,107.000,171.000,224.000,386.000,337.000,0.000,42.000,43778347.250,0.000,0.000,0.000,15640213.000,15286881.750,321.000,294.000
50%,20231.000,3120001.000,184854645.500,4585.000,138333420.000,37661145.000,6136944.500,34222861.000,35608985.500,49693963.000,82942230.000,74623180.000,160833.000,12158489.500,3400.000,1018.000,165.000,967.000,966.000,1287.000,2253.000,1937.500,12.000,438.000,168098750.000,1516748.500,3981946.500,76.000,68591283.500,62915461.000,1756.000,1620.000
75%,20232.000,3120186.000,670335090.750,24990.750,508560976.750,146729693.250,47316932.250,149821606.500,139409257.750,182213664.750,307190696.500,282747046.500,2212020.500,62126169.750,18704.250,5709.750,1965.000,6277.000,5095.000,6880.000,11848.000,11338.000,170.000,3233.750,610772422.500,23182793.500,50244980.000,1468.000,254483601.000,235029535.000,9660.000,8417.250
max,20232.000,3130327.000,995677000000.000,12090273.000,669799000000.000,485357000000.000,107916000000.000,242249000000.000,326543000000.000,386343000000.000,537243000000.000,304162000000.000,1549259735.000,90669553834.000,5827863.000,6262410.000,1474148.000,2611826.000,3245770.000,4287338.000,6708993.000,4615812.000,329040.000,3087414.000,841405000000.000,391742210636.000,126314287024.000,1514884.000,354347000000.000,411544000000.000,4519958.000,5329793.000


In [107]:
# 중복 확인
estimated_sales[estimated_sales.duplicated()]

,기준_년분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드_명,당월_매출_금액,당월_매출_건수,주중_매출_금액,주말_매출_금액,시간대_06~11_매출_금액,시간대_11~14_매출_금액,시간대_14~17_매출_금액,시간대_17~21_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,주중_매출_건수,주말_매출_건수,시간대_06~11_매출_건수,시간대_11~14_매출_건수,시간대_14~17_매출_건수,시간대_17~21_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,당월_개인_매출_금액,당월_법인_매출_금액,시간대_21~06_매출_금액,시간대_21~06_매출_건수,연령대_30_40_매출_금액,연령대_50_60_매출_금액,연령대_30_40_매출_건수,연령대_50_60_매출_건수


### 2-6-3. 분기별 데이터 연도 기준 변환

In [108]:
estimated_sales_year = estimated_sales.drop(columns = ['기준_년분기_코드','상권_구분_코드','상권_코드'])\
                                      .groupby(['상권_구분_코드_명', '상권_코드_명', '서비스_업종_코드_명'])\
                                      .mean()\
                                      .reset_index()

## 2-7. 데이터 병합

In [109]:
preprocessed_df = pd.merge(living_popul_year, resident_popul_year)
preprocessed_df = pd.merge(preprocessed_df, store_year)
preprocessed_df = pd.merge(preprocessed_df, working_popul_year)
preprocessed_df = pd.merge(preprocessed_df, facilitie_year)
preprocessed_df = pd.merge(preprocessed_df, estimated_sales_year)
preprocessed_df = pd.merge(preprocessed_df, area)

In [110]:
preprocessed_df

,상권_구분_코드_명,상권_코드_명,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,시간대_06_11_유동인구_수,시간대_11_14_유동인구_수,시간대_14_17_유동인구_수,시간대_17_21_유동인구_수,시간대_21_06_유동인구_수,주중_유동인구_수,주말_유동인구_수,연령대_30_40_유동인구_수,연령대_50_60_유동인구_수,총_상주인구_수,남성_상주인구_수,여성_상주인구_수,연령대_10_상주인구_수,연령대_20_상주인구_수,연령대_30_40_상주인구_수,연령대_50_60_상주인구_수,서비스_업종_코드_명,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수,총_직장_인구_수,집객시설수,교통시설수,당월_매출_금액,당월_매출_건수,주중_매출_금액,주말_매출_금액,시간대_06~11_매출_금액,시간대_11~14_매출_금액,시간대_14~17_매출_금액,시간대_17~21_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,주중_매출_건수,주말_매출_건수,시간대_06~11_매출_건수,시간대_11~14_매출_건수,시간대_14~17_매출_건수,시간대_17~21_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,당월_개인_매출_금액,당월_법인_매출_금액,시간대_21~06_매출_금액,시간대_21~06_매출_건수,연령대_30_40_매출_금액,연령대_50_60_매출_금액,연령대_30_40_매출_건수,연령대_50_60_매출_건수,엑스좌표_값,와이좌표_값,자치구_코드_명,행정동_코드_명,영역_면적
0,골목상권,4.19민주묘지역 2번,868030.000,361490.250,506539.250,128381.000,163223.250,169905.000,105148.000,103902.500,145960.750,343114.000,619012.250,249017.500,221593.250,354833.000,1247,596,651,135,121,324,667,미용실,3,3,0,0,0,0,0,96,0.000,0.000,34384167.000,1306.500,21903351.000,12480816.000,862554.500,9952889.750,10859155.000,12330743.000,20974955.000,13117808.250,136012.500,3947675.500,824.500,482.000,55.000,368.500,438.250,440.250,919.500,384.000,10.500,177.500,34092763.250,291403.750,378824.750,4.500,20604973.000,9404102.250,788.750,326.500,201112,461090,강북구,우이동,60794
1,골목상권,4.19민주묘지역 2번,868030.000,361490.250,506539.250,128381.000,163223.250,169905.000,105148.000,103902.500,145960.750,343114.000,619012.250,249017.500,221593.250,354833.000,1247,596,651,135,121,324,667,분식전문점,2,3,0,0,0,0,1,96,0.000,0.000,116484380.667,11618.333,73379411.667,43104969.000,2948848.333,24056213.667,28566904.667,46828332.333,48130947.000,67577039.667,2272384.000,24387549.667,7502.333,4116.000,175.333,2425.333,3058.333,4640.333,4431.333,7173.667,309.667,2723.000,115707986.667,776394.000,14084081.667,1319.000,52310936.000,36737117.000,5420.667,3152.000,201112,461090,강북구,우이동,60794
2,골목상권,4.19민주묘지역 2번,868030.000,361490.250,506539.250,128381.000,163223.250,169905.000,105148.000,103902.500,145960.750,343114.000,619012.250,249017.500,221593.250,354833.000,1247,596,651,135,121,324,667,일식음식점,3,3,0,0,0,0,0,96,0.000,0.000,18435727.000,1519.250,15543098.250,2892628.750,0.000,9658447.500,3074001.750,5623422.500,7005386.750,11430340.250,561557.250,6405609.500,1359.250,160.000,0.000,821.500,247.000,441.750,463.250,1056.000,55.250,601.250,18435727.000,0.000,79855.250,9.000,6061697.500,5406862.750,495.250,367.500,201112,461090,강북구,우이동,60794
3,골목상권,4.19민주묘지역 2번,868030.000,361490.250,506539.250,128381.000,163223.250,169905.000,105148.000,103902.500,145960.750,343114.000,619012.250,249017.500,221593.250,354833.000,1247,596,651,135,121,324,667,치킨전문점,1,3,0,0,0,0,2,96,0.000,0.000,66483629.000,2189.750,43158115.000,23325514.000,52763.750,1660777.000,5881497.500,31315232.250,40050256.500,22819818.250,208308.750,6765486.000,1372.000,817.750,1.250,81.750,189.250,1130.500,1291.500,851.250,11.500,244.000,62870074.750,3613554.250,27573358.500,787.000,24407459.000,31488820.500,835.500,1050.750,201112,461090,강북구,우이동,60794
4,골목상권,4.19민주묘지역 2번,868030.000,361490.250,506539.250,128381.000,163223.250,169905.000,105148.000,103902.500,145960.750,343114.000,619012.250,249017.500,221593.250,354833.000,1247,596,651,135,121,324,667,커피-음료,3,4,0,0,0,0,1,96,0.000,0.000,366772357.500,44223.750,229175262.500,137597095.000,51444290.750,68243920.000,75151055.250,91457383.750,170207345.250,193885402.500,6104389.500,90242333.750,29133.750,15090.000,6777.500,8375.250,9374.250,11000.250,19724.750,24327.250,978.000,12572.750,364092747.750,2679609.750,80475707.750,8696.500,160759334.500,106986690.250,18299.000,12201.750,201112,461090,강북구,우이동,60794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

# 3. 군집화 알고리즘

In [73]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.cluster import DBSCAN, KMeans

def categorization(df):
    col_num = list(df.select_dtypes(include = 'number').columns)

    for col in col_num:
        unique_bin_edges = np.unique(df[col])
        
        # 중복된 경계를 제거하고 qcut 적용
        _, bins = pd.qcut(df[col], 
                           q=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
                           labels=False,  # labels를 False로 지정하여 정수형 라벨 생성
                           duplicates='drop',
                           retbins=True)  # 경계만을 반환하도록 수정
        
        # 중복된 경계를 제거하고 cut 적용
        df[f'{col}_category'] = pd.cut(df[col], bins=bins, labels=False, include_lowest=True)
    
    df.drop(columns = col_num, inplace = True)

def clustering(df):
    ### 모델 학습 부분
    df_service = list(df['서비스_업종_코드_명'].unique())
    train = df[['서비스_업종_코드_명', '집객시설수', '당월_매출_금액', '유사_업종_점포_수', '교통시설수', '총_상주인구_수', '총_유동인구_수', '총_직장_인구_수']]
    df_length = 0
    #
    for i in df_service:
        temp = train[train['서비스_업종_코드_명'] == i]

        categorization(temp)
        
        scaler = StandardScaler()
        col_num = list(temp.select_dtypes(include = 'number').columns)
        temp[col_num] = scaler.fit_transform(temp[col_num])
        
        col_cat = list(temp.select_dtypes(include = 'object').columns)
        le = LabelEncoder()
        temp[col_cat] = temp[col_cat].apply(le.fit_transform)
        
        kmeans = KMeans(n_clusters=4, random_state=2023)
        kmeans.fit(temp)
        labels = kmeans.predict(temp)
        
        result = df[df['서비스_업종_코드_명'] == i]
        result['cluster'] = labels
        result.to_csv(f"../cluster_by_service/{i}.csv", index = False)

In [74]:
preprocessed_df.loc[preprocessed_df['서비스_업종_코드_명'] == '운동/경기용품', ['서비스_업종_코드_명']] = '운동경기용품'
clustering(preprocessed_df)